<img style="float: center;" src="images/CI_horizontal.png" width="1000">
<center>
    <span style="font-size: 1.5em;">
        <a href='https://www.coleridgeinitiative.org'>Website</a>
    </span>
</center>


<center> Julia Lane, Benjamin Feder, Tian Lou, Lina Osorio-Copete </center>

# Unsupervised Machine Learning

There are problems where there does not exist a target variable that we want to predict but we want to understand "natural" grouping or patterns in the data. Unsupervised machine learning methods are used to tackle these problems. Clustering is the most common unsupervised machine learning technique, but you might also be aware of neural networks or principal components analysis (PCA). This notebook will provide an introduction to unsupervised machine learning through a clustering example.

## Introduction to Clustering

Clustering is used to group data points together that are similar to each other. Optimally, a given clustering method will produce groupings with high intra-cluster (within) similarity and low inter-cluster (between) similarity. Clustering algorithms typically require a distance or similarity metric to generate clusters. They take a dataset and a distance metric (and sometimes additional parameters), and they generate clusters based on that distance metric. The most common distance metric used is Euclidean distance, but other commonly used metrics are Manhattan, Minkowski, Chebyshev, cosine, Hamming, Pearson, and Mahalanobis.

Most clustering algorithms also require the user to specify the number of clusters (or some other parameter that indirectly determines the number of clusters) in advance as a parameter. This is often difficult to do a priori and typically makes clustering an iterative and interactive task. Another aspect of clustering that makes it interactive is often the difficulty in automatically evaluating the quality of the clusters. While varioius analytical clustering metrics have been developed, the best clustering is task-dependent and thus must be evaluated by the user. There may be different clusterings that can be generated with the same data. You can imagine clustering similar news stories based on the topic content, based on the writing style or based on sentiment. The right set of clusters depends on the user and the task they have. Clustering is therefore typically used for exploring the data, generating clusters, exploring the clusters, and then rerunning the clustering method with different parameters or modifying the clusters (by splitting or merging the previous set of clusters). Interpreting a cluster can be nontrivial: you can look at the centroid of a cluster, look at frequency distributions of different features (and compare them to the prior distribution of each feature).

Here, we will focus on **K-Means clustering** (*k* defines the number of clusters), which is considered to be the most commonly used clustering method. The algorithm works as follows:
1. Select *k* (the number of clusters you want to generate).
2. Initialize by selecting k points as centroids of the *k* clusters. This is typically done by selecting k points uniformly at random.
3. Assign each point a cluster according to the nearest centroid.
4. Recalculate cluster centroids based on the assignment in **(3)** as the mean of all data points belonging to that cluster.
5. Repeat **(3)** and **(4)** until convergence.

The algorithm stops when the assignments do not change from one iteration to the next. The final set of clusters, however, depends on the starting points. If initialized differently, it is possible that different clusters are obtained. One common practical trick is to run *k*-means several times, each with different (random) starting points. The *k*-means algorithm is fast, simple, and easy to use, and is often a good first clustering algorithm to try and see if it fits your needs. When the data are of the form where the mean of the data points cannot be computed, a related method called *K-medoids* can be used.

### Learning Objectives

In this notebook, you will use *k*-means clustering to better understand Ohio's labor market in the third quarter of 2013. We've already developed a handful of employer-level measures in the [Data Preparation](ML_Data_Prep.ipynb) notebook. We will try a few different values of *k* to see how we can best understand the labor market by looking for differentiation between each of the clusters.

We are hoping to identify clusters with "optimal employers," and then use our cohort of 2012-13 Ohio community college graduates from previous notebooks to see who is finding employment in these better clusters. What industries do these "better" employers tend to come from? Are community college graduates finding jobs with these employers? Are there earnings outcomes what we would expect? Essentially, we will be using *k*-means clustering to help bolster our sense of the various associate's degrees to employment pathways, and which ones happen to be more successful than others.

## Import Packages and Set Up


In [ ]:
# pandas-related imports
import pandas as pd

# Numpy
import numpy as np

# database interaction imports
import sqlalchemy

# import viz 
import matplotlib.pyplot as plt

#import clustering
from sklearn.cluster import KMeans
from sklearn.preprocessing import scale
from sklearn.cluster import AgglomerativeClustering
from sklearn.mixture import GaussianMixture
import scipy.cluster.hierarchy as sch

In [ ]:
# to create a connection to the database, 
# we need to pass the name of the database and host of the database

host = 'stuffed.adrf.info'
DB = 'appliedda'

connection_string = "postgresql://{}/{}".format(host, DB)
conn = sqlalchemy.create_engine(connection_string)

## 1. Read in the Data

As mentioned above, we have created a DataFrame that contains employer IDs, industry codes, and 11 employer characteristics and saved it in a .csv file. Let's use `read_csv` to import the data to a DataFrame.

In [ ]:
# Import data
emp = pd.read_csv('employer_all_new.csv', index_col = 0)

In [ ]:
# Let's take a look at the DataFrame
emp.head()

> If you do not use `index_col = 0` when you read in the .csv file, you should see an extra column `Unnamed:0`, which contains the row numbers as an additional column. The extra column is generated when running `to_csv`, which will morph a given DataFrame into a .csv file. We need to remove it when we read in the data. Alternatively, you can set the `index` argument to `False` to avoid creating this additional column.

In [ ]:
# Basic Statistics
emp.describe()

### Clean the Data

We need to remove the `employer` variable from our DataFrame since these features do not provide any explanatory power for our k-means algorithm. Additionally, k-means algorithms only work properly with continuous features. This is because k-means calculates its distance measure using euclidean distance, which is the distance between each data point and the centroid of a cluster. It is hard to assign positions for categorical variables in the euclidean space. Thus, we also need to remove `naics_3_digit` from `emp`.

> There are more complicated clustering algorithms that do not use Euclidean distances and thus allow categorical variables in the model. If you are interested in them, you can take a look at: <a href='https://pypi.org/project/kmodes/'>k-modes</a> and <a href='https://thinkdatascience.com/post/2019-12-16-introduction-python-package-gower/'>gower</a>.

In [ ]:
# get rid of employer and naics for clustering
emp_ml = emp.drop(['employer', 'naics_3_digit'], axis = 1)

In [ ]:
#Take a look at the DataFrame now. It only has the 11 employer characteristics. 
#We will use them as the features in the k-means clustering.
emp_ml.head()

In [ ]:
# make sure every column is numeric
emp_ml.info()

## 2. Choose the Number of Clusters, *K*

Running a *k*-means model is simple: we just need to use `KMeans()` and choose the number of clusters `n_clusters`. What number should we choose? Here, we have 11 features, so it is hard to visualize the data and decide the proper number by using our eyes. Let's start with a small number, such as 3, and see how the results look like.  

Because *k*-means clustering will generate different results (due to different starting points), we will set a seed so that the work in this notebook can be reproducible using the `random_state` argument in `KMeans()`. To get the same results, you must use the same seed before running the clustering algorithm every time. Luckily, if you set the same seed as your collaborators and are running the same *k*-means algorithm, you will see the same results, even if you are working in different environments, i.e. Jupyter notebooks.

> There are additional parameters in the `KMeans()` function that you can adjust and tune them based on your assumptions and knowledge about the sample and the model.

### k = 3

In [ ]:
# Run the model
kmeans = KMeans(n_clusters = 3, random_state = 1)

Now we can fit the model on `emp_ml`. **It is important that we scale the features** before we compute *k*-means clustering, especially because the metrics are on a variety of numerical scales. Here, we can just use `scale(emp_ml)`.

In [ ]:
# Fit the model 
kmeans.fit(scale(emp_ml))

In [ ]:
# Predict which cluster each employer belongs to
y_means = kmeans.predict(scale(emp_ml))

Now you have run your first *k*-means clustering model! Let's put the results and features into a DataFrame, `frame_3`, so that we can analyze the attributes of employers in each cluster.

In [ ]:
# Put employer IDs, industry codes, and the eleven features into frame_3.
frame_3 = pd.DataFrame(emp)

# Add the predicted clusters into frame_3 as well.
frame_3['cluster'] = y_means

# Let's see how many employers are in each cluster.
frame_3['cluster'].value_counts()

We can see that most of the employers are concentrated in cluster 1. In the perfect world, we would want them to be distributed more evenly across clusters, but in some cases, it may make sense that they wouldn't. Most importantly, we are looking for high intra-cluster similarity and low inter-cluster similarity.

Are there major differences in the characteristics of employers in each cluster? We have created a function `characteristics_table` for you. It tells you whether the cluster average of a variable is higher **(+)** or lower **(-)** than the full sample average of that variable. To use the function, you need two arguments: 1) `cols`, a list of employer characteristics you want to look at; 2) `cframe`, the DataFrame you have created in the previous step, `frame_3`, which contains employer characteristics and the cluster each employer belongs to. The function returns a DataFrame.

In [ ]:
# First, we want to include the number of employees within each cluster in our tables so that we do not 
# need to go back and check the numbers every time when we look at the table.

# Here, the mini function get_counts returns a DataFrame that has the cluster indicators and number of 
# employers within each cluster.

def get_counts(frame):
    c_frame = frame.groupby(['cluster'])['employer'].agg('count').reset_index().rename(columns={"employer":"n"})
    return c_frame

def characteristics_table(cols,cframe):
    
    """
    Creates an employer characteristics comparison table.
    If the mean of a cluster is greater than the mean of all employers, labelled by '+'.
    If the mean of a cluster is less than the mean of all employers, labelled by '-'.
    
    Parameters
    ----------
    cols: a list of employer characteristics
    cframe: a DataFrame contains the employer characteristics listed in cols 
            and the predicted cluster of each employer
    """
    
    cframe_no_cols = cframe[cols] 
    
    all_df = get_counts(cframe)
    
    for var in cols:
        all_df = all_df.merge((cframe.groupby(['cluster'])[var].agg('mean') > 
                              cframe[var].mean()).reset_index(), on = 'cluster')
        
    all_df[cols] = all_df[cols].replace({False:'-',True:'+'})
    
        
    return all_df

In [ ]:
# Create a list of employer characteristics
# Here, we just include all the features we used for the clustering model
cols = emp_ml.columns.tolist()

# Run the function and save the results of the function to a DataFrame
c3_emp_charcs = characteristics_table(cols,frame_3)

# Let's see how the table looks like!
c3_emp_charcs

We can see that our biggest cluster, *cluster 1*, contains relatively small employers that pay their employees low wages and have high job turnover rates. *cluster 0* also has relatively small employers, but on average, they pay their employees more and have a lower job turnover rate. *cluster 2* are high-quality, larger employers.

However, in Ohio, many big companies have a large number of low-wage workers and high job turnover rates, such as big grocery stores and supermarket franchises. Clearly, our three-cluster model did not properly capture these companies. Let's increase k to see if it improves our results.

We will set k=7 and see how the employer distributions across clusters change.

> The Quarterly Census of Employment and Wages (QCEW) data has even more detailed information about employers, such as whether an employer is single unit or multi-unit, the location of the employer, and the monthly employment, etc. Some of these variables are crucial for the clustering model. For example, we might want to treat each unit of a multi-unit employer as a separate employer or treat single- and multi-unit employers differently, because different sites of a multi-site employer may have different industries sometimes. Amazon Inc. headquarters may be listed in Information sector (NAICS code: 51), while one of its warehouses in Ohio may be in the Retail Trade sector (NAICS code: 45). As another example, Davis and Haltiwanger (1992) pointed out that job desctruction and creation rates are different for single-site and multi-site employers. Because of these potential differences between single-site and multi-site employers, when we are running the clustering model, they may end up in different clusters.

### k = 7

In [ ]:
# Run the model
kmeans = KMeans(n_clusters = 7, random_state = 42)

# Fit the model
kmeans.fit(scale(emp_ml))

# Predict which cluster each employer belongs to
y_means = kmeans.predict(scale(emp_ml))

# Put employer IDs, industry codes, and the eleven features into frame_7
frame_7 = pd.DataFrame(emp)

# Add the predicted clusters into frame_7 as well
frame_7['cluster'] = y_means

# Let's see how many employers are in each cluster
frame_7['cluster'].value_counts()

In [ ]:
# Again, let's see if we get more diversed clusters with the employer
cols = emp_ml.columns.tolist()

# Run the function and save the results of the function to a DataFrame
c7_emp_charcs = characteristics_table(cols,frame_7)

# Let's see how the table looks like!
c7_emp_charcs

In the seven-cluster model, the employers are distributed more evenly across clusters. We see three clusters of high-quality big employers (high earnings, many full-quarter employees, low turnover rates), one cluster of high-quality small employers, one cluster of average-quality small employers, and two clusters of low-quality small employers. Although we find more diversified clusters, the information conveyed in our +/- table is not enough for us to tell more differences across clusters. In a later section, we will create more robust employer tables and use boxplots to get better inter-cluster details.

In the context of this problem, do you think seven is the optimal amount of clusters? If not, how many clusters should we choose? We can use the *Elbow method* to select the optimal cluster number. Recall that *k*-means starts with k random cluster centers (centroids), assigns each data point to the closest centroid, and calculates the distances between each point and the centroid. Then it moves the positions of the centroids and repeats the previous steps until there is convergence. In the *Elbow method*, we try different k values and calculate the sum of squared errors (`SSE`) after the model converges. Then we plot all the `SSE` by K in a line-chart. The line-chart should resemble an arm.

### Elbow Curve

In [ ]:
# Let's plot the Elbow Curve!

sse = [1] # sum of squared error
delta = [0] # change in SSE
percent = [0] # percentage change in SSE 

# Run the K-Means Clustering model for K= 1 to 15
for cluster in range(1, 15):
    kmeans = KMeans(n_clusters = cluster, random_state=42)
    kmeans.fit(scale(emp_ml))
    sse.append(kmeans.inertia_)
    delta.append(kmeans.inertia_ - sse[cluster-1])
    percent.append(delta[cluster]/sse[cluster-1])

#save the results in a DataFrame
sse_df = pd.DataFrame({'Cluster':range(1,15), 'SSE':sse[1:],'Delta':delta[1:],'Percent':percent[1:]})

In [ ]:
#Plot the Elbow Curve
plt.plot(sse_df['Cluster'], sse_df['SSE'], marker = 'o')
plt.xticks(np.arange(0,15,step = 2))
plt.yticks(np.arange(25000,2000000,step = 400000))
plt.ylabel('Sum of Squared Errors', fontsize = 15)
plt.xlabel('Cluster Number, K', fontsize = 15)
plt.title('Elbow Curve', fontsize = 20);

We can see that SSE decreases as we increase k. Here, it decreases faster when k is small. As k increases, the reduction in SSE becomes smaller. We try to choose the number around the `inflection point`, where the change in SSE becomes negligible, indicating that there is little room to improve the model by increasing k. On our graph, the elbow curve becomes flat around 10.

We can also look at the specific changes in SSE. In the previous step, we saved the SSE, the change in SSE (Delta), and 
the percentage change in SSE (percent) in `sse_df`.

> Using the elbow curve can be entirely subjective. For instance, here, you can have a reasonable argument for 8 clusters.

In [ ]:
# Take a look at the DataFrame. Do you have the same conclusion?
sse_df

<font color=red><h3> Checkpoint 1: Run a K-Means clustering model </h3></font> 

1. Take a look at the elbow curve and `sse_df`, which number(s) do you think is (are) optimal?

2. Choose a cluster number that you think is best (other than 3,7, and 10). Use `Kmeans(n_cluster = , random_state = )` to run a k-means clustering model with the number you choose. Save your results and features in `frame_k`.

3. Create the +/- employer characteristics table `ck_emp_charcs` with the function `characteristics_table(cols,frame_k)`.

4. Compare your results with the results we got previously. Do you find any differences? Are the results improved?


## 3. Employer Characteristics Within Each Cluster

In this section, we will run the model with what we've determined to be the optimal cluster number, 10, and answer the following questions:
- What are the characteristics of employers within each cluster?
- What are the most common industries in each cluster?
- Which cluster(s) of employers are "good" employers? Which cluster(s) of employers are "bad" employers?

Let's run the 10-cluster model and save all the features and predicted clusters in `frame_10`.

### k = 10

In [ ]:
# Run the model
kmeans = KMeans(n_clusters = 10, random_state = 42)

# Fit the model
kmeans.fit(scale(emp_ml))

# Predict which cluster each employer belongs to
y_means = kmeans.predict(scale(emp_ml))

# Put employer IDs, industry codes, and the eleven features into frame_10
frame_10 = pd.DataFrame(emp)

# Add the predicted clusters into frame_10 as well
frame_10['cluster'] = y_means

# Let's see how many employers are in each cluster
frame_10['cluster'].value_counts()

In [ ]:
# Save all features to a list
cols = emp_ml.columns.tolist()

# Run the function and save the results of the function to a DataFrame
c10_emp_charcs = characteristics_table(cols,frame_10)

# Let's see how the table looks like!
c10_emp_charcs

In [ ]:
# We can also use .groupby and .agg to look at means and standard deviation of
# employer characteristics within each cluster
c10_emp_desc = frame_10.groupby(['cluster'])[cols].agg(['mean','std']).T.round(1)

c10_emp_desc

### NAICS codes

Recall that we did not include industry codes in the clustering model because categorical variables are not suitable for k-means clustering. However, now that we have ran the model, we can use industries to help describe the characteristics of employers in each cluster. Here, we will use the 2-digit industry code.

In [ ]:
#Convert 3-digit industry codes to 2-digit
frame_10['naics_2_digit'] = frame_10['naics_3_digit'].str[0:2]

#Check what we get
frame_10['naics_2_digit'].unique()

Let's get their descriptions from the lookup table `oh_naics2_codes_lkp`.

In [ ]:
# get naics code lookup table
# note that NAICS lookup code is numeric. We need to convert it to string by using astype(str)
naics_lkp = pd.read_sql('select * from data_ohio_olda_2018.oh_naics2_codes_lkp',conn).astype(str)

In [ ]:
# Check the values of 2-digit NAICS code in the lookup table
naics_lkp.naics2.unique()

Note that some of the 2-digit NAICS codes are not in the lookup table. This is because some sectors are represented by a range of 2-digit codes. For example, manufacturing are 31-33, retail trade are 44-45, and transporation and warehousing are 48-49. Our lookup table only has one code for each of the three sectors. Therefore, we need to make some changes to the 2-digit NAICS codes in `frame_10` so they'll match the NAICS codes in the lookup table, which we can do using the `replace()` function.

In [ ]:
# convert 2 digit NAICS code
frame_10.replace(to_replace={"naics_2_digit": {'32': '31', '33':'31', '45':'44', '49':'48'}},inplace=True)

In [ ]:
# merge with the lookup table to get description
frame_10 = frame_10.merge(naics_lkp, left_on='naics_2_digit',right_on='naics2', how = 'left')

In [ ]:
# Confirm our table looks right
frame_10[['naics_3_digit','naics_2_digit','naics2','naics2_desc']].head(5)

### Most popular NAICS by cluster

Now we can see the NAICS codes that are most present by industry!

In [ ]:
# Group by cluster and 2-digit industry
c10_ind_counts = frame_10.groupby(['cluster','naics_2_digit','naics2_desc'])['employer'].agg(['count']).reset_index()

# see c10_ind_counts
c10_ind_counts

In [ ]:
# Sort values from the highest to the lowest within each cluster
c10_ind_counts = c10_ind_counts.sort_values(by=['cluster','count'], ascending=False)

#see c10_ind_counts
c10_ind_counts

In [ ]:
# Keep the top 3 industries within each cluster
c10_top3_ind_counts = c10_ind_counts.groupby(['cluster']).head(3)

Just to make the table a bit more readable, we will add a `rank` column to differentiate between the most popular industries by cluster.

In [ ]:
# create a rank column
c10_top3_ind_counts['rank'] = c10_top3_ind_counts.groupby(['cluster']).cumcount()+1

In [ ]:
# Rename the column names so it is easier for your audience to understand
c10_top3_ind_counts = c10_top3_ind_counts.rename(columns = {'naics_2_digit':'ind_code',
                                                            'naics2_desc':'ind_desc',})

In [ ]:
c10_top3_ind_counts.head()

We can combine `c10_top3_ind_counts` with our employer characteristics table, `c10_emp_charcs` to get a grasp of the different clusters.

In [ ]:
# We can use a loop to combine c10_emp_charcs with the top three industries 

for n in [1,2,3]:
    c10_emp_charcs = c10_emp_charcs.merge(c10_top3_ind_counts[c10_top3_ind_counts['rank']==n], 
                                          on = 'cluster', how = 'outer')
    
    c10_emp_charcs = c10_emp_charcs.rename(columns = {'ind_code': 'ind_code_top{}'.format(n),
                                                      'ind_desc': 'ind_desc_top{}'.format(n), 
                                                      'count':'ind_count_top{}'.format(n)})

c10_emp_charcs = c10_emp_charcs.drop(['rank_x','rank_y','rank'], axis = 1)

In [ ]:
c10_emp_charcs.T

The top 3 industries help us learn more about the employers in each cluster. For example, from the characteristics table `c10_emp_charcs`, we know that clusters 8 and 9 are both big firms with many high-wage and stable employees. But the top 3 industries in clusters 8 and 9 are somewhat different, which could be even more significant if we analyze 3-digit NAICS codes.

### Fuzzy boxplots

To see more details about employers in each cluster, we can also use boxplots to visualize employer characteristics within each cluster. We have created the function `get_boxplot_stats` for you. Due to privacy and confidentiality concerns, we cannot export normal boxplots from ADRF. Instead, we need to use a safe boxplot, which has fuzzy min, max, median, etc. The function `get_boxplot_stats` returns the statistics you need for the boxplot.

In [ ]:
def get_boxplot_stats(frame,var,k):
    
    """
    Returns stats and xticklabels for the safe boxplot.
    
    Parameters
    ----------
    frame: the DataFrame that contains the employer characteristics and the predicted cluster 
    of each employer
    var : the variable you want to plot
    k: the list of clusters you want to plot
    """
    
    fuzzy_25 = (frame.groupby(['cluster'])[var].quantile(.2) + frame.groupby(['cluster'])[var].quantile(.3))/2
    fuzzy_med = (frame.groupby(['cluster'])[var].quantile(.45) + frame.groupby(['cluster'])[var].quantile(.55))/2
    fuzzy_75 = (frame.groupby(['cluster'])[var].quantile(.7) + frame.groupby(['cluster'])[var].quantile(.8))/2
   
    outlier_min = fuzzy_25 - 1.5*(fuzzy_75-fuzzy_25)
    outlier_max = fuzzy_75 + 1.5*(fuzzy_75-fuzzy_25)
    
    df_min = pd.DataFrame([0])
    df_max = pd.DataFrame([0])
    stats = []
    xticklabel = []
    
    for n in k:
        df = frame[(frame['cluster']==n) & (frame[var] > outlier_min[n]) & 
                   (frame_3[var] < outlier_max[n])][var].unique()

        if df.size == 0:
            df_min['c{}'.format(n)] = fuzzy_25
            df_max['c{}'.format(n)] = fuzzy_75
        else:
            df_min['c{}'.format(n)] = (sorted(df)[0] + sorted(df)[1])/2
            df_max['c{}'.format(n)]= (sorted(df)[-1] + sorted(df)[-2])/2

        stats.append ({'med': fuzzy_med[n], 'q1': fuzzy_25[n], 
                                'q3': fuzzy_75[n], 'whislo': df_min['c{}'.format(n)][0], 
                                'whishi': df_max['c{}'.format(n)][0]})
        xticklabel.append(n)
    
    return stats,xticklabel
    

In [ ]:
# Use a list to save the cluster you want to look at
cluster_to_viz = [0,1,2,3,4,5,6,7,8,9]

# Get stats and clusters from the get_boxplot_stats() function
stats, xticklabel = get_boxplot_stats(frame_10,'num_employed',cluster_to_viz)

_, ax = plt.subplots()
ax.bxp(stats, showfliers=False)
ax.set_xticklabels(xticklabel)

#set the title
ax.set_title('Number of employees during 2013Q3\nby Cluster')

#Create axes same as before
ax.set_xlabel("Cluster")
ax.set_ylabel("Number of employees");

From the numerical summary table, we can get the same conclusion that clusters 3,8,and 9 consist of big companies, but we would not have been able to see that cluster 3's employers have especially high number of employees. Companies in cluster 9 are bigger than companies in cluster 8 on average. How about other measures, such as earnings per employee?

In [ ]:
# Use a list to save the cluster you want to look at
cluster_to_viz = [0,1,2,3,4,5,6,7,8,9]

# Get stats and clusters from the get_boxplot_stats() function
stats, xticklabel = get_boxplot_stats(frame_10,'avg_earnings',cluster_to_viz)

_, ax = plt.subplots()
ax.bxp(stats, showfliers=False)
ax.set_xticklabels(xticklabel)

#set the title
ax.set_title('Average earnings per employee \nby Cluster')

#Create axes same as before
ax.set_xlabel("Cluster")
ax.set_ylabel("Average earnings");

The pattern of average earnings are very different from that of number of employees. Employees of cluster 5 firms mostly earn extremely high quarterly incomes. While employers in clusters 8 and 9 pay their employees more on average than employers in clusters 1,2,4, and 7, they also have some low-income workers. We can further analyze earning characteristics, particularly for those more relevant to entry-level job applicants from community colleges, by looking at the bottom 25 percentile earnings of employers in each cluster.

In [ ]:
# Use a list to save the cluster you want to look at
cluster_to_viz = [0,1,2,3,4,5,6,7,8,9]

# Get stats and clusters from the get_boxplot_stats() function
stats,xticklabel = get_boxplot_stats(frame_10,'bottom_25_pctile',cluster_to_viz)
_, ax = plt.subplots()
ax.bxp(stats, showfliers=False)
ax.set_xticklabels(xticklabel)

#set the title
ax.set_title('Bottom 25 Percentile Earnings by Cluster')

#Create axes same as before
ax.set_xlabel("Cluster")
ax.set_ylabel("Bottom 25 Percentile Earnings");

Do high average earnings imply that an employer is "good"? Not necessarily. An employer could have a few high-wage management level workers and many low-wage workers but still ends up with high average earnings. The bottom 25 percentile earnings allow us to have a deeper understanding about the workers that have lower incomes within a firm. For example, the average earnings of clusters 2 and 4 are about the same, but the bottom 25 percentile earnings of cluster 2 employers are lower, implying employers in cluster 2 are likely to be of a lower quality pay-wise than that of cluster 4 employers. 

> We can do the same practice for the other employer-level measures. Just keep in mind, if you want to export these boxplots, **please drop clusters that have less than 10 employers**. This will be discussed further in the [Disclosure Review](03_Disclosure_Review.ipynb) notebook.

<font color=red><h3> Checkpoint 2: Examine employer characteristics of your model </h3></font> 

1. Given the `frame_k` you have created in checkpoint 1, find out the top 3 industries within each cluster. You can use 3-digit or 2-digit industry codes.

2. Create boxplots for `avg_earnings`, `full_quarter_avg_earnings`, and `top_25_pctile`.


## 4. How are 2012-13 Ohio Community College Graduates distributed amongst our clusters?

At this point, you should have a decent grasp of Ohio's employers during 2013Q3. Next, we will focus on our cohort of interest, 2012-13 Ohio community college students, and their employers by using our previous clustering results. Specifically, in this section, we will explore:

- How are the 2012-13 Ohio community college graduates distributed amongst the 10 clusters?
- How are the employers of 2012-13 Ohio community college graduates distributed amongst the 10 clusters?
- What are average earnings (from primary employers) of graduates in each cluster?
- What are the top 3 industries of graduates in each cluster?
- What are the top 3 degrees of graduates in each cluster?

Recall that in the [Data Exploration](02_1_Data_Exploration.ipynb) notebook, we joined our cohort table with `oh_ui_wage_by_employer` to create `cohort_oh_jobs_emp`. This table contains employment records of our cohort during the first year after graduation. For those with multiple employers during a quarter, we have only kept the employers that paid them the most during that quarter. 

Let's pull that table from database and limit it to records from the third quarter of 2013.

### Our Cohort in 2013Q3

In [ ]:
# Pull 2012-13 Ohio Community College graduates and the employer that paid them the most
qry = '''
select * from ada_20_osu.cohort_oh_jobs_emp
where job_date='2013-07-01';
'''

cc_emp_df = pd.read_sql(qry,conn)

In [ ]:
# see cc_emp_df
cc_emp_df.head()

In [ ]:
# Let's see how many people in our cohort were employed during 2013 Q3.
cc_emp_df.shape

Now let's merge `cc_emp_df` with `frame_10` based on each person's primary employers and see how our cohort and their employers distribute across clusters.

In [ ]:
# Merge tables and only keep the variables that we will use for analysis 
cc_df = frame_10[['employer','cluster']].merge(cc_emp_df[['ssn_hash','employer','wages','naics_3_digit']], 
                                               on = 'employer', how='right')

In [ ]:
# see cc_df
cc_df.head()

### Cohort's Earnings Outcomes by Cluster

From here, we can see where our cohort landed and if those in our cohort experiencing similar earnings outcomes across the clusters to the general Ohio workforce population in 2013Q3.

In [ ]:
# 2012-13 Community college graduates' distribution
cc_cluster = cc_df.groupby(['cluster'])['ssn_hash'].nunique().reset_index()

cc_cluster

In [ ]:
# 2012-13 community college graduates' employers' distribution
cc_emp_df = cc_df.groupby(['cluster'])['employer'].nunique().reset_index()

# see cc_emp_df
cc_emp_df

In [ ]:
# Total number of employers within each cluster
c10_counts = get_counts(frame_10).rename(columns = {'n':'cluster_n'})

# see c10_counts
c10_counts

In [ ]:
#combine the three DataFrames
cc_cluster = cc_cluster.merge(cc_emp_df, on = 'cluster')
cc_cluster = cc_cluster.merge(c10_counts, on = 'cluster', how = 'outer')

#rename columns to be more descriptive
cc_cluster = cc_cluster.rename(columns = {'ssn_hash':'stud_n','employer':'emp_n'})

# see cc_cluster
cc_cluster

Let's add more columns to this table. Specifically, we will calculate:

1. The percentage of employers in (1) that are in each cluster `emp_per`
2. The percentage of 2012-13 Ohio community college graduates in each cluster `stud_per`
3. The average earnings (from primary employers) of graduates in each cluster `avg_earnings`

In [ ]:
# Percent of employers in each cluster
cc_cluster['emp_per'] = ((cc_cluster['emp_n'] / cc_cluster['emp_n'].sum())*100).round(2).astype(str) + '%'

# Percent of graduates in each cluster
cc_cluster['stud_per'] = ((cc_cluster['stud_n'] / cc_cluster['stud_n'].sum())*100).round(2).astype(str) + '%'

#get the average earnings
avg_earn = cc_df.groupby(['cluster'])['wages'].agg(['mean']).reset_index()

cc_cluster = cc_cluster.merge(avg_earn, on = 'cluster', how = 'outer')
cc_cluster = cc_cluster.rename(columns = {'mean':'avg_earnings'})

cc_cluster

We can see that REDACTED of these graduates that were employed in 2013Q3 are hired by employers in the largest cluster, cluster 7. Their employers account for REDACTED of the employers that hired at least one member of our cohort. The average earnings of graduates in this cluster are the third lowest among all the clusters at REDACTED dollars per quarter, which translates to REDACTED dollars per year. Based on the graphs and tables we generated previously, cluster 7 consists of primarily small employers that pay their employees low wages. However, job turnover rates for these employers are relatively low when compared with employers in other clusters. 

The second and third most popular clusters for our cohort clusters 8 and 9, which contain fewer employers than all the other clusters, except for cluster 3. The two clusters account for only REDACTED of the employers that hired someone in our cohort, but they hired REDACTED of those in our cohort employed in 2013Q3. However, the average earnings of graduates in these two clusters are not much higher than graduates in cluster 7. This is consistent with the average earnings and bottom 25 percentile earnings boxplots.

Graduates in clusters 1 and 5 have the highest average earnings. Again, this is consistent with what the boxplots show: the average earnings and bottom 25 percentile earnings are the highest in clusters 1 and 5. However, these graduates make up less than REDACTED
of our cohort that found employment in 2013Q3.

> Note that the average earnings in this table are average earnings from the primary employers. These may understate some people's earnings because some of them may have worked for more than one employers during 2013Q3. However, if they have multiple jobs and each job does not pay them much, this does not affect the conclusion that some of them were working for low-quality employers after graduation.

> One fact our model cannot capture is that some community college graduates may have gone to a four-year college during the time period we are looking at. Thus, they are either not in the labor market or have part-time jobs which pay them little and may not match their skillset. 

> Note that no members of our cohort were hired by employers in cluster 6. Why do you think this may be the case? Refer back to the employers table.

### Cohort-specific NAICS by cluster

Next, we can find the top 3 industries for our cohort by cluster. To do so, we need to repeat the steps we did previously: 1) convert three-digit industry codes to two-digits; 2) make changes to the special two-digit industry codes; 3) join it with the lookup table to get indsutry description.

In [ ]:
# Convert 3-digit NAICS codes to 2-digit NAICS codes
cc_df['naics_3_digit'] = cc_df['naics_3_digit'].fillna("missing").astype(str)
cc_df['naics_2_digit'] = cc_df['naics_3_digit'].str[0:2]

In [ ]:
# Change the codes in Manufacturing, Retail Trades, and Transportation and Warehouse
cc_df.replace(to_replace={"naics_2_digit": {'32': '31', '33':'31', '45':'44', '49':'48'}},inplace=True)

In [ ]:
#Get the NAICS codes' descriptions from the lookup table
cc_df = cc_df.merge(naics_lkp, left_on='naics_2_digit',right_on='naics2',how='outer')

In [ ]:
cc_df.head()

In [ ]:
# Group by cluster and 2-digit NAICS codes
cc_top3_ind = cc_df.groupby(['cluster','naics2','naics2_desc'])['ssn_hash'].agg(['count']).reset_index()

# Sort counts from the highest to the lowest within each cluster
cc_top3_ind = cc_top3_ind.sort_values(by=['cluster','count'], ascending=False)

# Get the top 3 industries
cc_top3_ind = cc_top3_ind.groupby(['cluster']).head(3)

# Add the rank so it is easier for us to see in the table
cc_top3_ind['rank'] = cc_top3_ind.groupby(['cluster']).cumcount()+1

cc_top3_ind.head()

In [ ]:
#combine our main table with the top three industries 

for n in [1,2,3]:
    cc_cluster = cc_cluster.merge(cc_top3_ind[cc_top3_ind['rank']==n], on = 'cluster', how = 'outer')
    cc_cluster = cc_cluster.rename(columns = {'naics2': 'naics2_top{}'.format(n),
                                              'naics2_desc': 'naics2_desc_top{}'.format(n), 
                                              'count':'ind_count_top{}'.format(n)})

cc_cluster = cc_cluster.drop(['rank_x','rank_y','rank'], axis = 1)

In [ ]:
cc_cluster

### Cohort's Most Popular Degrees by Cluster

Finally, we can also see the most popular degrees by cluster. To do so, we need to recreate `cc_grads_recent`, which has the degree records of 2012-13 Ohio community college students and is limited to the most recent degree for students who have earned multiple degrees during 2012-13 academic year.

In [ ]:
# Get 2012-13 Ohio community college graduates

qry = '''
drop table if exists cc_grads;
create temp table cc_grads as
select a.*, lkp.*
from data_ohio_olda_2018.oh_hei_long a
left join data_ohio_olda_2018.oh_hei_campus_county_lkp lkp
on a.degcert_campus = lkp.campus_num
where ((a.degcert_yr_earned = '2012' and (a.degcert_term_earned = '4' or a.degcert_term_earned = '1')) or 
    (a.degcert_yr_earned = '2013' and (a.degcert_term_earned = '2' or a.degcert_term_earned = '3'))) and 
    lkp.campus_type_code in ('TC', 'SC', 'CC') and a.degcert_subject!='TRAMOD';
'''
conn.execute(qry, conn)

In [ ]:
# Keep the most recent degree for students who have earned more than one degrees
qry = '''
drop table if exists cc_grads_recent;
create temp table cc_grads_recent as
select distinct on (ssn_hash) *, left(degcert_subject, 2) as subject
from (
SELECT *, 
    CASE WHEN degcert_term_earned = 4 THEN
        format('%%s-%%s-01', degcert_yr_earned, 7)::date 
    WHEN degcert_term_earned = 1 THEN
        format('%%s-%%s-01', degcert_yr_earned, 10)::date 
    WHEN degcert_term_earned = 2 THEN
        format('%%s-%%s-01', degcert_yr_earned, 1)::date 
    WHEN degcert_term_earned = 3 THEN
        format('%%s-%%s-01', degcert_yr_earned, 4)::date 
    END AS deg_date
    from cc_grads
) q
order by ssn_hash, deg_date DESC
'''
conn.execute(qry)

Next, we need to convert 6-digit subject codes to 2-digit ones to get their descriptions from the lookup table `oh_subject_codes_lkp_new`.

In [ ]:
# Get degrees and degree descriptions
qry = '''
with cc_subject as (select ssn_hash, left(right('0'||degcert_subject,6),2) from cc_grads_recent)
select * from cc_subject
left join (select distinct(subject_code_2010_2),subject_desc_2010_2
      from data_ohio_olda_2018.oh_subject_codes_lkp_new) lkp
on cc_subject.left=lkp.subject_code_2010_2; 
'''

cc_subject = pd.read_sql(qry,conn)

In [ ]:
cc_subject.head()

In [ ]:
# Now, let's add the degree information to cc_df
cc_df = cc_df.merge(cc_subject, on = 'ssn_hash', how = 'left')

In [ ]:
cc_df.head()

Finally, let's find out the top 3 degrees for students in each cluster and add it to `cc_cluster`.

In [ ]:
# Group by cluster and subject codes to get counts
cc_top3_cip = cc_df.groupby(['cluster','subject_code_2010_2','subject_desc_2010_2'])['ssn_hash'].agg(['count']).reset_index()

# see cc_top3_cip
cc_top3_cip.head()

In [ ]:
# Sort the counts from highest to lowest within each cluster
cc_top3_cip = cc_top3_cip.sort_values(by=['cluster','count'], ascending=False)

# see cc_top3_cip
cc_top3_cip.head()

In [ ]:
# Get the top 3 degrees with in each cluster
cc_top3_cip = cc_top3_cip.groupby(['cluster']).head(3)

# see cc_top3_cip
cc_top3_cip.head()

In [ ]:
# create a rank column
cc_top3_cip['rank'] = cc_top3_cip.groupby(['cluster']).cumcount()+1

# see cc_top3_cip
cc_top3_cip.head()

In [ ]:
# Calculate percent of students with the top 3 degrees within each cluster
cc_top3_cip['degree_per'] = ((cc_top3_cip['count']/(cc_cluster['stud_n'].sum()))*100).round(2).astype(str) + '%'

# see cc_top3_cip
cc_top3_cip.head()

In [ ]:
cc_top3_cip = cc_top3_cip.rename(columns = {'subject_code_2010_2':'degree_code',
                                            'subject_desc_2010_2':'degree_desc',})

# see cc_top3_cip
cc_top3_cip.head()

In [ ]:
#combine our main table with the top three degrees 

for n in [1,2,3]:
    cc_cluster = cc_cluster.merge(cc_top3_cip[cc_top3_cip['rank']==n], on = 'cluster', how = 'outer')
    cc_cluster = cc_cluster.rename(columns = {'degree_code': 'degree_code_top{}'.format(n),
                                              'degree_desc': 'degree_desc_top{}'.format(n), 
                                              'count':'degree_count_top{}'.format(n),
                                              'degree_per':'degree_per_top{}'.format(n)})

cc_cluster = cc_cluster.drop(['rank_x','rank_y','rank'], axis = 1)

In [ ]:
cc_cluster.T

### Putting it all Together

Now let's take a closer look at our cohort and the clusters again. 

The top 3 industries and the top 3 degrees in each cluster are dominated by the REDACTED industry and graduates with degrees in REDACTED. This is not suprising. Recall that in the [Data Exploration](01_2_Dataset_Exploration.ipynb) notebook, we found that the degree with the most graduates is REDACTED. Some of them ended up in "high-quality" employers clusters (such as cluster 1), but others ended up in "low-quality" employers cluster (such as cluster 7). This may be related to the variations of REDACTED workers' skill levels and entry level earnings. For example, according to Bureau of Labor Statistics, the median annual wage of REDACTED is only REDACTED, while a REDACTED can expect to earn REDACTED at an entry-level position.

Similarly, when we look at graduates in clusters 7,8,and 9, the top 3 degrees are the same: REDACTED, REDACTED, and REDACTED. However, cluster 7 graduates are more likely to find employment in industries with many low-skill and low-income jobs, while some of the cluster 9 graduates work at more profitable industries, such as REDACTED.

Among graduates in the two high-quality employer clusters, REDACTED are amongst the Top 3 degrees. This is consistent with the high expected earnings of people with REDACTED degrees.

In conclusion, in our model, graduates in some degrees are more spread out across clusters, especially those in REDACTED, while students with other degrees, such as REDACTED, are more likely to find jobs with "high-quality" employers. Note that we have only used 2-digit industry codes and 2-digit subject codes. We may have a better understanding of our cohorts' employment if we use more detailed industry codes and subject codes. However, you need to make sure that the number of people and number of employers in each subgroup are more than 10 if you would like to export these results outside of ADRF.

<font color=red><h3> Checkpoint 3: Find out labor market and education characteristics for 2012-13 community college graduates </h3></font> 

1. Given the `frame_k` you have created in checkpoint 1, find out how many 2012-13 community college students are in each cluster of your model.

2. What are the average earnings (from primary employers) for graduates in each cluster of your model?

3. What are the top 3 degrees of graduates in each cluster of your model? You can use 2-digit or 4-digit subject codes. 


## References

Davis, Steven J., and John Haltiwanger. "Gross job creation, gross job destruction, and employment reallocation." *The Quarterly Journal of Economics* 107, no. 3 (1992): 819-863.

Foster, Ian, Rayid Ghani, Ron S. Jarmin, Frauke Kreuter, and Julia Lane, eds. *Big data and social science: A practical guide to methods and tools.* crc Press, 2016.

Elka, Torpey. "Employment outlook for occupations requiring an associate's degree, certificate or some college." *Career Outlook*, U.S. Bereau of Labor Statistics, November 2018.